In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.mevotech.com/part/' + input_.loc[a, 'Part_Number'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                list_tr = html.xpath('//div[@class="atwm-table-continer"]/table/tbody/tr')
                for tr in list_tr:
                    vehicle_temp = tr.xpath('./td[1]/text()')[0].strip() + ' '
                    vehicle_temp += tr.xpath('./td[2]/text()')[0].strip() + ' '
                    if '-' in tr.xpath('./td[3]/text()')[0]:
                        vehicle_temp += tr.xpath('./td[3]/text()')[0].split('-')[1].strip() + '-' + tr.xpath('./td[3]/text()')[0].split('-')[0].strip() + ' '
                    else:
                        vehicle_temp += tr.xpath('./td[3]/text()')[0].strip() + ' '
                    vehicle_temp += tr.xpath('./td[4]/text()')[0].strip()

                    list_vehicle.append(vehicle_temp)

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                         'Vehicle': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(17):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：221

[ok] - CMK80011
[尝试次数：1] - [剩余数量：204] - [当前时间：15:47:39]

[ok] - CMK100003
[尝试次数：1] - [剩余数量：203] - [当前时间：15:47:39]

[ok] - CMK7427
[尝试次数：1] - [剩余数量：202] - [当前时间：15:47:40]

[ok] - CMK7211
[尝试次数：1] - [剩余数量：201] - [当前时间：15:47:40]

[ok] - CMK80038
[尝试次数：1] - [剩余数量：200] - [当前时间：15:47:40]

[ok] - CMK5313
[尝试次数：1] - [剩余数量：199] - [当前时间：15:47:40]

[ok] - CMK7425
[尝试次数：1] - [剩余数量：198] - [当前时间：15:47:41]

[ok] - CMK5311
[尝试次数：1] - [剩余数量：197] - [当前时间：15:47:42]

[ok] - CMK7318
[尝试次数：1] - [剩余数量：196] - [当前时间：15:47:42]

[ok] - CMK100004
[尝试次数：1] - [剩余数量：195] - [当前时间：15:47:43]

[ok] - CMK3198
[尝试次数：1] - [剩余数量：194] - [当前时间：15:47:43]

[ok] - CMK80053
[尝试次数：1] - [剩余数量：193] - [当前时间：15:47:44]

[ok] - CMK80054
[尝试次数：1] - [剩余数量：192] - [当前时间：15:47:46]

[ok] - CMK80055
[尝试次数：1] - [剩余数量：191] - [当前时间：15:47:46]

[ok] - CMK80115
[尝试次数：1] - [剩余数量：190] - [当前时间：15:47:46]

[ok] - CMK80325
[尝试次数：1] - [剩余数量：189] - [当前时间：15:47:47]

[ok] - CMK80052
[尝试次数：1] - [剩余数量：188] - [当前时间：15:47:47]

[ok] - CMK80324
[尝试次数：1] - 

[ok] - CMK90418
[尝试次数：1] - [剩余数量：58] - [当前时间：15:48:43]

[ok] - CMK90419
[尝试次数：1] - [剩余数量：57] - [当前时间：15:48:45]

[ok] - CMK90382
[尝试次数：1] - [剩余数量：56] - [当前时间：15:48:45]

[ok] - CMK90446
[尝试次数：1] - [剩余数量：55] - [当前时间：15:48:45]

[ok] - CMK90420
[尝试次数：1] - [剩余数量：54] - [当前时间：15:48:45]

[ok] - CMK90265
[尝试次数：1] - [剩余数量：53] - [当前时间：15:48:45]

[ok] - CMK90421
[尝试次数：1] - [剩余数量：52] - [当前时间：15:48:45]

[ok] - CMK90423
[尝试次数：1] - [剩余数量：51] - [当前时间：15:48:45]

[ok] - CMK90448
[尝试次数：1] - [剩余数量：50] - [当前时间：15:48:46]

[ok] - CMK90497
[尝试次数：1] - [剩余数量：49] - [当前时间：15:48:46]

[ok] - CMK90494
[尝试次数：1] - [剩余数量：48] - [当前时间：15:48:47]

[ok] - CMK90665
[尝试次数：1] - [剩余数量：47] - [当前时间：15:48:47]

[ok] - CMK90498
[尝试次数：1] - [剩余数量：46] - [当前时间：15:48:49]

[ok] - CMK80726
[尝试次数：2] - [剩余数量：45] - [当前时间：15:48:49]

[ok] - CMK90489
[尝试次数：1] - [剩余数量：44] - [当前时间：15:48:50]

[ok] - CMK9052
[尝试次数：1] - [剩余数量：43] - [当前时间：15:48:51]

[ok] - CMK90508
[尝试次数：1] - [剩余数量：42] - [当前时间：15:48:52]

[ok] - CMK90506
[尝试次数：1] - [剩余数量：41] - [当前时间：15:4